In [1]:
import tensorflow as tf
import keras
#from tensorflow.python import keras
from tensorflow.python.framework import graph_io
from tensorflow.python.keras._impl.keras.models import load_model
from tensorflow.python.keras import backend as K
from keras.utils.generic_utils import CustomObjectScope
#from tensorflow.python.keras._impl.keras.utils.generic_utils import CustomObjectScope
from tensorflow.python.keras._impl.keras.applications import mobilenet
#from tensorflow.python.keras._impl.keras.applications import mobilenet as mb
#from tensorflow.python.keras._impl.keras.applications.mobilenet import relu6
#from  tensorflow.python.keras._impl.keras.applications.mobilenet import relu6
#from keras.layers import DepthwiseConv2D, ReLU

# Clear any previous session.
tf.keras.backend.clear_session()

save_pb_dir = './models'
model_fname = './models/mobileTrain.h5'


def relu6(x):
  return K.relu(x, max_value=6)


# def relu6(x):
#    relu6 = ReLU(6.)
#    return relu6(x)
#    #return K.relu(x, max_value=6)
    

with CustomObjectScope({"relu6": keras.layers.ReLU(6.)}):
    keras_mobilenet= tf.keras.applications.mobilenet.MobileNet(weights='imagenet')
    keras_mobilenet.compile(optimizer=tf.keras.optimizers.Adam(),
                          loss='mean_squared_error',
                          metric='accuracy')
    mobilenet_estimator = tf.keras.estimator.model_to_estimator(keras_model=keras_mobilenet)
    
# from keras.utils.generic_utils import CustomObjectScope

# with CustomObjectScope({'relu6': relu6}):
#     model = load_model('./models/mobileTrain.h5')
    
    
def freeze_graph(graph, session, output, save_pb_dir='.', save_pb_name='frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen

# This line must be executed before loading Keras model.
tf.keras.backend.set_learning_phase(0) 

model = load_model(model_fname)

session = tf.keras.backend.get_session()

input_names = [t.op.name for t in model.inputs]
output_names = [t.op.name for t in model.outputs]

# Prints input and output nodes names, take notes of them.
print(input_names, output_names)

frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs], save_pb_dir=save_pb_dir)


print("success")

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarning: Passing (type, 1) or 

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_save_checkpoints_secs': 600, '_task_id': 0, '_global_id_in_cluster': 0, '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_save_checkpoints_steps': None, '_task_type': 'worker', '_num_ps_replicas': 0, '_model_dir': '/tmp/tmp8zxdrjqr', '_tf_random_seed': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0447223940>, '_log_step_count_steps': 100, '_session_config': None, '_num_worker_replicas': 1, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_save_summary_steps': 100, '_train_distribute': None}


ValueError: Unknown activation function:relu6